In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Seaborn for ploting data
import seaborn as sns
import matplotlib.pyplot as plt

# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf

from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import re
import spacy
import nltk
from nltk.corpus import stopwords, words
import re
import spacy
import nltk
from nltk.corpus import stopwords, words, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

import torch
from torch.utils.data import Dataset

from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction

#Import CSV

In [21]:
# Load the data from the uploaded CSV files
#data = pd.read_csv('drive/MyDrive/merged_affiliations.csv')
#authors_df = pd.read_csv('drive/MyDrive/merged_authors.csv')
data = pd.read_csv('drive/MyDrive/merged_researches.csv')

# Display the first few rows of each DataFrame to understand their structure
#(affiliations_df.head(), authors_df.head(), researches_df.head())
data.dropna(inplace = True, subset=['title', 'abstract'])
data.head()

,Unnamed: 0,id,doi,eid,cover_date,title,abstract,subject_areas,auth_keywords,authors_id,citedby_count,ref_count,ref_ids,published_year,published_month,published_day
1,1,85060936020,10.23919/PIERS.2018.8597669,2-s2.0-85060936020,2018-12-31,Flexible Printed Active Antenna for Digital Te...,"© 2018 The Institute of Electronics, Informati...",MATE|ENGI,NaN,57192376216|6507497381,1.0,4,85006043726|85046336244|85060914424|85046368249,2018,12,31
2,2,85052201238,10.1016/j.ces.2018.08.042,2-s2.0-85052201238,2018-12-31,Parametric study of hydrogen production via so...,© 2018 Elsevier LtdComputational fluid dynamic...,ENGI|CHEM|CENG,NaN,57202924002|7004487886|25923304100|50662017700,21.0,42,2942655685|84908055658|85052199786|84859716773...,2018,12,31
3,3,85051498032,10.1016/j.apsusc.2018.08.059,2-s2.0-85051498032,2018-12-31,Superhydrophobic coating from fluoroalkylsilan...,© 2018 Elsevier B.V. A superhydrophobic/supero...,MATE|CHEM|PHYS,NaN,24074703800|57190429582|7403123085|7401969567|...,37.0,45,78349312523|53249093621|84980335769|8486252720...,2018,12,31
4,4,85050678366,10.1016/j.aca.2018.07.045,2-s2.0-85050678366,2018-12-31,Electrochemical impedance-based DNA sensor usi...,© 2018 Elsevier B.V. A label-free electrochemi...,BIOC|CHEM|ENVI,NaN,56524669400|6506927536|6602082849|8532633300|6...,68.0,55,56249149272|33749077283|0037834610|73949151264...,2018,12,31
5,5,85059846549,10.17512/pjms.2018.18.2.24,2-s2.0-85059846549,2018-12-30,Evaluation of outsourcing transportation contr...,"© 2018, Czestochowa University of Technology. ...",BUSI,NaN,6504603029,15.0,20,54349103890|0033235290|69649096182|85043470598...,2018,12,30


In [24]:
# Download NLTK stopwords and load SpaCy's English-language model
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#nltk.download('words')
nltk.download('punkt')
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def get_wordnet_pos(treebank_tag):
    """Converts treebank tags to wordnet tags."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
     # Lemmatizer
    lemmatizer = WordNetLemmatizer()

    #specific remove word
    extra_words_to_remove = ['thailand', 'elsevier', 'b','v','ieee','springer', 'pte', \
                             'international', 'publishing', 'ag', 'phitsanulok','0','1'\
                             ,'ltd','ltdthe','association','mdpi','licensee','basel','switzerland'\
                             ,'japan', 'russia', 'cambodia', 'laos', 'vietnam', 'australia', 'singapore'\
                             ,'pte','ios', 'walter', 'de', 'gruyter', 'gmbh', 'esterified', 'ieeethis'\
                             ,'nowaieeedays', 'ieeeasset', 'thai', 'switzerland', 'iop', '2019a', 'hci'\
                             'ra','c']
    extra_words_to_remove += ['using','can','used','also','however', 'not', 'could', 'non', 'may'
            , 'many', 'total', 'amount', 'one', 'two', 'three','thus']
    extra_words_to_remove += [str(i) for i in range(0,977)]
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove year numbers
    s = re.sub(r'\b\d{4}\b', '', s)
    # Remove stopwords
    stop_words = set(stopwords.words('english')) - {'not', 'can'}
    stop_words.update(extra_words_to_remove)
    s = " ".join([word for word in s.split() if word not in stop_words])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    # Tokenization and POS Tagging
    tokens = nltk.word_tokenize(s)
    tagged_tokens = pos_tag(tokens)

    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]
    # Remove trailing whitespace
    s = " ".join(lemmatized_tokens).strip()
    return s


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
# Apply the cleaning function to the title and abstract columns
data['title'] = data['title'].apply(text_preprocessing)
data['abstract'] = data['abstract'].apply(text_preprocessing)
cleaned_data = data[['title', 'abstract']]

In [26]:
#to CSV
cleaned_data.to_csv('cleaned_titles_abstracts.csv', index=False)